In [2]:
import pandas as pd
import teradatasql
import getpass
import pickle
import re
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
from nltk.corpus import stopwords
#from nltk.corpus import stopwords
pd.set_option("display.max_columns", None)
#PW=getpass.getpass()
query = """ Select Specific_Site,Description_of_Incident,Injury_Date 
From zods_kpi.oji_dtl_num 
Where Injury_Date Between Date '2018-01-01' And Date '2021-12-31'
And Employee_Division='ACS'
And OSHA_Recordability = 'OSHA Recordable'"""
#query = """ select * from zods_kpi.oji_dtl_num """
with teradatasql.connect(None, host = "dwprod.delta.com", user = '476196', password = PW, logmech = 'LDAP') as connect:
    df = pd.read_sql(query, connect)
#df.to_excel('ACS_Injury_Description.xlsx',index=False)

········


C:\Users\476196\AppData\Local\Temp\1\ipykernel_23944\2674823275.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connect)


In [29]:
# load the model from disk
loaded_model = pickle.load(open('pipeline_model.pkl', 'rb'))

In [30]:
def clean_description(description):    
    # Remove all the special characters
    description = re.sub(r'\W', ' ', description)    
    # remove all single characters
    description = re.sub(r'\s+[a-zA-Z]\s+', ' ', description)    
    # Remove single characters from the start
    description = re.sub(r'\^[a-zA-Z]\s+', ' ', description)     
    # Substituting multiple spaces with single space
    description = re.sub(r'\s+', ' ', description, flags=re.I)    
    # Removing prefixed 'b'
    description = re.sub(r'^b\s+', '', description)    
    # Converting to Lowercase
    description = description.lower()    
    # Lemmatization
    description = description.split()
    description = [stemmer.lemmatize(word) for word in description]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    description = [w for w in description if not w in stop_words]
    description = ' '.join(description)    
    return description

In [38]:
x_test = df["Description_of_Incident"].apply(clean_description)
test_predictions = loaded_model.predict(x_test)
df['Pred_Category'] = test_predictions
df['Pred_Category'] = df['Pred_Category'].map({1:'Ergo',2:'Others'})
df.to_excel('Injury_Triage_2018to21.xlsx',index=False)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4651 entries, 0 to 4650
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Specific_Site            4651 non-null   object
 1   Description_of_Incident  4651 non-null   object
 2   Injury_Date              4651 non-null   object
dtypes: object(3)
memory usage: 109.1+ KB


In [39]:
df.head()

,Specific_Site,Description_of_Incident,Injury_Date,Pred_Category
0,"Jetway, Interior, Stairs",While closing the forward passenger cabin door...,2021-03-14,Others
1,Ramp,"On June 9th, 2021 I was ALA in charge working ...",2021-06-09,Ergo
2,On A/C at Ramp,Dan Clay was lifting bags and felt discomfort ...,2021-05-20,Ergo
3,Ramp,EE states she was pulling an empty bag cart ar...,2019-03-01,Ergo
4,Bagroom Outbound,EE states that he was repositioning a bag cart...,2020-09-20,Ergo


In [40]:
df.Pred_Category.value_counts()

Others    2373
Ergo      2278
Name: Pred_Category, dtype: int64

In [44]:
df[df['Pred_Category']=='Ergo'].sort_values(by='Injury_Date')

,Specific_Site,Description_of_Incident,Injury_Date,Pred_Category
1688,On A/C at Ramp,"After working in the bid, agent felt pain in h...",2018-01-02,Ergo
4336,Ramp,"EE WAS HOOKING UP TRANSPORTERS/ULDS, AS HE WAS...",2018-01-02,Ergo
4203,Bagroom Outbound,EE was loading baggage into the carts in the F...,2018-01-02,Ergo
3799,Ramp,EE was in the AFT bin of flight 2026 loading b...,2018-01-02,Ergo
3620,Inside Cargo Bin,Agent was off loading a flight on a MD90 - tos...,2018-01-03,Ergo
...,...,...,...,...
4486,Bagroom Outbound,Employee was loading bags onto a cart and felt...,2021-12-30,Ergo
492,Ramp,Agent reported that he was off loading MCI-MSP...,2021-12-30,Ergo
3117,Ramp Other,"On the last flight I was assigned on 12/30/21,...",2021-12-30,Ergo
47,Inside Cargo Bin,EE states discomfort in the R shoulder from pu...,2021-12-31,Ergo


In [45]:
#df[df['Pred_Category']=='Ergo'].sort_values(by='Injury_Date').to_excel('Injury_Triage_18to21.xlsx',index=False)

In [3]:
df_full=df.copy()
df_full.head()

,Reference_Nbr,Injury_Date,EE_Rpt_Date,Submitted_Date,Date_of_Hire,Date_to_Recordable,Employee_Nbr,Employee_Name,Employee_Division,Report_Division,Employee_Station,Employee_Department,FOP_Base,FOP_Fleet,Employee_Job_Title,YOS,YOS_Partial,EE_Age,Employee_Sex,Supervisor_Name,Location_of_Incident,Specific_Site,Area_Bay_Gate_Shop_or_Zone,Description_of_Incident,Primary_Inj_Type,Primary_Inj_Body_Part,Primary_Injury_Side_of_Body,Contributing_Equipment_Object,Contributing_Equipment_Other,Direct_Cause,Task_Being_Performed,Related_Act_Condition,Work_Related,Reported_Event_Descriptor,EE_Start_Time,Incident_Time,Consecutive_Days_Worked,Overtime_Swap,Employee_on_TDY,TDY_Station,Flight_Oper_Carrier,Flight_Origin_Date,Flight_Number,Flight_Sched_Origin,Flight_Sched_Destination,Flight_Act_Origin,Flight_Act_Destination,AC_Owner,AC_Ship_Number,AC_Fleet,Turbulence_Injury,Investigation_Summary,OSHA_Recordability,Reason_for_Recordability,Nbr_LWD,Nbr_of_TD_Days,Investigation_Status,Recordable_Status,IFS_Category,TOP_Category,Investigation_Status_Calc,Risk_Severity_Nbr,Risk_Probability_Nbr,Reported_in_24Hrs,Days_to_Recordable,Months_to_Recordable,DL_OJI_Count,GIR_Count,DART_Count,DAFWII_Count,CV19_Count,CV19_Rec_Count,Minor_Inj_Count,Moderate_Inj_Count,Major_Inj_Count,Critical_Inj_Count,FORM_DATA_CLOSED_DATE,CSI_Form_Severity,Report_Station,isr_num,top_bu,top_bu_id,top_lmtc_region,top_inj_type,empl_lag,lead_lag,rcd_lag,imcr_pt,non_metric,effective_start,effective_end,Risk_Severity_Desc
0,INJ/25229-15,2015-12-26,2015-12-26,2015-12-26,None,None,061241100,"SCHOENLEB,UTE M",ACS,ACS,ORD,125,None,None,BW CSA RFT,NaN,NaN,45.0,Female,"OSBORNE, JESSICA F",None,"Jetway, Interior, Stairs",Gate/Ticket/Terminal,Employee was walking in jetway at E16 and when...,Strained Muscle/Body Part,Knee,Right,"Walking Surface (Floor, Ground)",None,Walking,retired:Transporting Bags to/from Bridge Loader,Walking,None,Migration Injury Reports,04:00:00,09:20:00,NaN,No,No,None,None,None,None,None,None,None,None,DL,None,A/C Not a Contributing Factor,None,None,Non-Recordable,None,0.0,0.0,closed,Non Recordable,None,,Closed,NaN,NaN,1,NaN,NaN,1,0,0,0,0,0,0,0,0,0,None,None,ORD,0,,,,,0,0,0,0,0,2020-11-23,2099-12-31,None
1,INJ/66845-19,2019-11-04,2019-11-05,2019-11-06,2016-02-22,None,033156900,"MILLER,STEPHANIE,ANNE",IFS,IFS,DTW,610,None,None,FLIGHT ATTENDANT,3.0,3.7,29.0,Female,"LYNCH,AUSTIN,M",Employer,Galley,OnBoard Aircraft,"The FA pulled the beverage cart out, extremely...",Strained Muscle/Body Part,"Back, Lower",Bi-Lateral,Cart/Full Beverage,None,Pulling,Providing Beverage Service,Improper Body Mechanics,Yes,Injury/Illness onboard flight,06:21:00,12:00:00,2.0,No,No,None,DL,2019-11-04,2884,LGA,BOS,LGA,BOS,DL,009528,B717,No,"The FA pulled the beverage cart out, extremely...",Non-Recordable,None,0.0,0.0,closed,Non Recordable,Ergonomic,,Closed,1.0,4.0,1,NaN,NaN,1,0,0,0,0,0,0,0,0,0,None,None,DTW,0,,,,,0,0,0,0,0,2021-04-21,2099-12-31,Negligible
2,INJ/25712-15,2015-09-24,2015-09-25,2015-09-28,None,2015-10-06,079377000,"DURKEE, VIOLET",IFS,IFS,JFK,610,None,None,Flight Attendant,NaN,NaN,53.0,Female,"dawkins, simone",None,Galley,OnBoard Aircraft,THE FASTEN SEAT BELT LIGHT CAME ON DURING DECE...,Other Occupational Diseases,"Back, Lower",Left,"Cart, Meal",None,Struck By,Preparing Cabin for Departure/Arrival,Equipment Not Secured,None,Migration Injury Reports,00:00:00,09:00:00,NaN,No,No,None,DL,None,0410,JFK,FRA,None,None,DL,174,B767,None,None,International Recordable,Restricted/Transitional Duty or Lost Work Days,106.0,0.0,closed,Recordable,None,,Closed,3.0,NaN,1,12.0,0.0,1,1,1,1,0,0,0,1,0,0,None,None,JFK,10,,,,,0,1,1,2,0,2020-11-23,2099-12-31,Moderate
3,INJ/12446-14,2014-05-09,2014-05-13,2014-05-15,None,2014-06-03,070497200,"YOCKEY,KAY L",IFS,IFS,JFK,610,None,None,FLIGHT ATTENDANT,NaN,NaN,49.0,Female,"SAUERWINE, MARK",None,Aisle,OnBoard Aircraft,EE states a cart fell over on EE right foot.,Contusion/Bruise,Foot,Right,Beverage Cart,None,Dropped On,Providing Beverage Service,Distraction/Inatten

In [7]:
#df_full[df_full['Recordable_Status'] =='Recordable'].info()
df_full[df_full['OSHA_Recordability'] =='OSHA Recordable'].info()
#df_full['OSHA_Recordability'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31449 entries, 7 to 82607
Data columns (total 92 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Reference_Nbr                  31449 non-null  object 
 1   Injury_Date                    31449 non-null  object 
 2   EE_Rpt_Date                    30719 non-null  object 
 3   Submitted_Date                 31449 non-null  object 
 4   Date_of_Hire                   8751 non-null   object 
 5   Date_to_Recordable             31449 non-null  object 
 6   Employee_Nbr                   31449 non-null  object 
 7   Employee_Name                  31448 non-null  object 
 8   Employee_Division              31449 non-null  object 
 9   Report_Division                31449 non-null  object 
 10  Employee_Station               31448 non-null  object 
 11  Employee_Department            31445 non-null  object 
 12  FOP_Base                       753 non-null   

In [8]:
df_full['OSHA_Recordability'].value_counts()

Non-Recordable                  41279
OSHA Recordable                 31449
International Non-Recordable     5528
International Recordable         4314
Under Investigation                39
Name: OSHA_Recordability, dtype: int64

In [10]:
df_full['Recordable_Status'].value_counts()

Non Recordable    46846
Recordable        35763
Name: Recordable_Status, dtype: int64

In [17]:
query = """ Select *
From zods_kpi.oji_dtl_num 
Where Injury_Date Between Date '2018-01-01' And Date '2021-12-31'
And Employee_Division='ACS'"""
with teradatasql.connect(None, host = "dwprod.delta.com", user = '476196', password = PW, logmech = 'LDAP') as connect:
    df_2018 = pd.read_sql(query, connect)
df_2018.info()
#And OSHA_Recordability = 'OSHA Recordable'

C:\Users\476196\AppData\Local\Temp\1\ipykernel_23944\1090134844.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2018 = pd.read_sql(query, connect)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11814 entries, 0 to 11813
Data columns (total 92 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Reference_Nbr                  11814 non-null  object 
 1   Injury_Date                    11814 non-null  object 
 2   EE_Rpt_Date                    10626 non-null  object 
 3   Submitted_Date                 11814 non-null  object 
 4   Date_of_Hire                   5277 non-null   object 
 5   Date_to_Recordable             4674 non-null   object 
 6   Employee_Nbr                   11814 non-null  object 
 7   Employee_Name                  11813 non-null  object 
 8   Employee_Division              11814 non-null  object 
 9   Report_Division                11814 non-null  object 
 10  Employee_Station               11813 non-null  object 
 11  Employee_Department            11810 non-null  object 
 12  FOP_Base                       0 non-null     